In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
embeddings = {}
with open('glove.6B.50d.txt',encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coeffs = np.asarray(values[1:],dtype='float32')
        
        #print(word)
        #print(coeffs)
        embeddings[word] = coeffs

In [3]:
df = pd.read_csv("combined.csv")


/home/srishti/anaconda3/envs/ml/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (6,7,12,13,14,15,16,17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.head()

,Unnamed: 0,airline,date_published,title,text,date_flown,rating,verified_user,route,aircraft,traveller_type,cabin_flown,seat_comfort,cabin_staff_service,food_beverages,ground_service,inflight_entertainment,value_for_money,wifi_and_connectivity,recommended
0,1,adria-airways,2017-12-05,"""should be ashamed of their operations""","| Ljubljana to Vienna. Overall, the flight s...",September 2017,3,True,Ljubljana to Vienna,ATR-72,Solo Leisure,Economy Class,2,4,1,1,NaN,3,NaN,no
1,2,adria-airways,2017-11-20,"""Two nice short flights""",| Two nice short flights from Ljubljana to Sa...,November 2017,9,True,Ljubljana to Sarajevo,CRJ700 / ATR72,Business,Economy Class,5,5,3,5,NaN,3,NaN,yes
2,3,adria-airways,2017-10-27,"""extremely bad service""",| We were 11 people coming in with a short co...,October 2017,2,True,Zurich to Ljubjana,NaN,Couple Leisure,Economy Class,3,3,NaN,1,NaN,1,NaN,no
3,4,adria-airways,2017-09-16,"""never fly this airline again""",| Ljubljana to Munich after arriving from At...,June 2017,2,True,Ljubljana to Munich,NaN,Family Leisure,Economy Class,3,4,1,1,NaN,2,1,no
4,5,adria-airways,2017-04-19,"""can't remember a flight delay""",| Ljubljana to Zurich on April 18th. Adria Ai...,April 2017,9,True,Ljubljana to Zurich,CR9,Business,Economy Class,5,5,4,5,4,4,NaN,yes


In [5]:
df = df[['text','recommended']]

In [6]:
df.head()

,text,recommended
0,"| Ljubljana to Vienna. Overall, the flight s...",no
1,| Two nice short flights from Ljubljana to Sa...,yes
2,| We were 11 people coming in with a short co...,no
3,| Ljubljana to Munich after arriving from At...,no
4,| Ljubljana to Zurich on April 18th. Adria Ai...,yes


In [7]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70284 entries, 0 to 70283
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   text         70284 non-null  object
 1   recommended  70284 non-null  object
dtypes: object(2)
memory usage: 1.1+ MB


In [9]:
import re

In [10]:
data =df
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70284 entries, 0 to 70283
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   text         70284 non-null  object
 1   recommended  70284 non-null  object
dtypes: object(2)
memory usage: 1.1+ MB


In [11]:
data['text'] = data['text'].str.lower()

In [12]:
import nltk
from nltk.corpus import stopwords

In [13]:
stop  = set(stopwords.words('english'))

In [14]:
# types = []
# for i in range(data.shape[0]):
#     #print(type(data['text'][i]))
#     #np.append(types,type(data['text'][i]))
#     types.append(type(data['text'][i]))
#     #if isinstance(data['text'][i], !str):
#         #print(i)


In [15]:
# set(types)

In [16]:
#data.drop(data.index[51469])
#data['text'][51469]

In [17]:
# for i in range(len(types)):
#     if types[i]=='str':
#         print(i)

In [18]:
for i in range(0,70284,1):
    if i%5000 == 1:
        print(i)
    words =  nltk.word_tokenize(data['text'][i])
    words = [w.lower() for w in words if len(w)>2 and w not in stop]
    data['text'][i] = words

1
5001
10001
15001
20001
25001
30001
35001
40001
45001
50001
55001
60001
65001
70001


In [19]:
# import gensim
# from gensim.models import word2vec
# from gensim.models import KeyedVectors

In [20]:
#word_vectors = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin',binary=True)

In [21]:
#type(word_vectors)

In [22]:
#type(data['text'][2])

In [23]:
from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer() 


In [24]:
for i in range(70284):
    if i%5000 == 0:
        print(i)
    words = data['text'][i]
    wv = []
    for w in words:
        w = lemmatizer.lemmatize(w)
        if embeddings.get(w) is None:
            continue
        wvec = embeddings[w]
        wv.append(wvec)
    if len(wv) == 0:
        data['text'][i] = None
        
    else:
        data['text'][i] = wv
        

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000


In [25]:
sizes=[]
for i in range(70284):
    if data['text'][i] is None:
        print(i)
        continue
    sizes.append(len(data['text'][i]))

5168
12223
12224
20094
20095
22016
23093
27491
27493
28896
29190
30801
31523
34302
40006
40914
41983
42880
48968
49486
50888
51322
59733
62355
63976


In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70284 entries, 0 to 70283
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   text         70259 non-null  object
 1   recommended  70284 non-null  object
dtypes: object(2)
memory usage: 1.1+ MB


In [27]:
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70259 entries, 0 to 70258
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   text         70259 non-null  object
 1   recommended  70259 non-null  object
dtypes: object(2)
memory usage: 1.1+ MB


In [28]:
np.array(sizes)

array([195,  70,  40, ...,  18,  15,  78])

In [29]:
np.min(sizes)

1

In [30]:
X = data.iloc[0:,0]


In [31]:
Y = data.iloc[0:,1]

In [32]:
#X = np.array(X)

In [33]:
#np.save('dataX.npy', X)

In [34]:
(unique, counts) = np.unique(np.array(sizes),return_counts=True)

In [35]:
frequencies = np.asarray((unique, counts)).T
print(frequencies)

[[   1    5]
 [   5    1]
 [   6    2]
 [   7    7]
 [   8   10]
 [   9   37]
 [  10   57]
 [  11   83]
 [  12  122]
 [  13  165]
 [  14  203]
 [  15  242]
 [  16  264]
 [  17  312]
 [  18  356]
 [  19  412]
 [  20  435]
 [  21  531]
 [  22  539]
 [  23  637]
 [  24  712]
 [  25  733]
 [  26  824]
 [  27  894]
 [  28  878]
 [  29  952]
 [  30 1046]
 [  31 1080]
 [  32 1002]
 [  33 1090]
 [  34 1109]
 [  35 1005]
 [  36 1066]
 [  37 1076]
 [  38  990]
 [  39 1016]
 [  40 1038]
 [  41 1089]
 [  42 1112]
 [  43 1031]
 [  44 1059]
 [  45  994]
 [  46 1016]
 [  47  965]
 [  48 1000]
 [  49  989]
 [  50  946]
 [  51  910]
 [  52  876]
 [  53  874]
 [  54  890]
 [  55  910]
 [  56  814]
 [  57  795]
 [  58  824]
 [  59  840]
 [  60  761]
 [  61  762]
 [  62  766]
 [  63  728]
 [  64  711]
 [  65  718]
 [  66  771]
 [  67  689]
 [  68  664]
 [  69  693]
 [  70  654]
 [  71  635]
 [  72  615]
 [  73  574]
 [  74  532]
 [  75  590]
 [  76  527]
 [  77  496]
 [  78  503]
 [  79  469]
 [  80  445]

In [36]:
import keras

Using TensorFlow backend.
/home/srishti/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/srishti/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/srishti/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/srishti/.local/lib/python3.6/site-packages/tensorflow/py

In [37]:
X2 = keras.preprocessing.sequence.pad_sequences(X, maxlen=200, padding='post',truncating = 'post',dtype = 'float32')

In [41]:
np.save("DataGloveY.npy",Y)

In [38]:
from keras.layers import *
from keras.models import Sequential

In [39]:
model = Sequential()
model.add
model.add(LSTM(64,input_shape=(10,300),return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(64,input_shape=(10,300)))
model.add(Dropout(0.3))
model.add(Dense(5))
model.add(Activation('sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 64)            93440     
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
Total params: 126,789
Trainable params: 126,789
Non-trainable params: 0
________________________________________________